# American Community Survey - California - 2017 Analysis

In this notebook, I will practice conducting statistical tests and visualizing data within Python. 

The data I'm using comes from the 2017 American Community Survey, from California only. 

In [8]:
# Libraries
import pandas as pd

## Data Preparation

I will import and clean up the data.

In [4]:
# Import data file
raw_data = pd.read_csv('psam_p06.csv')

In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377575 entries, 0 to 377574
Columns: 286 entries, RT to PWGTP80
dtypes: float64(85), int64(198), object(3)
memory usage: 823.9+ MB


In [7]:
print(raw_data.head(3))

  RT       SERIALNO  DIVISION  SPORDER  PUMA  REGION  ST   ADJINC  PWGTP  \
0  P  2017000000014         9        1  6703       4   6  1011189     59   
1  P  2017000000014         9        2  6703       4   6  1011189     57   
2  P  2017000000014         9        3  6703       4   6  1011189     54   

   AGEP  ...  PWGTP71  PWGTP72  PWGTP73  PWGTP74  PWGTP75  PWGTP76  PWGTP77  \
0    45  ...       64       63       55       17      100       65       62   
1    39  ...       51       54       59       19       95       53       56   
2    15  ...       50       62       52       14       99       53       60   

   PWGTP78  PWGTP79  PWGTP80  
0       68       17      101  
1       62       18       97  
2       60       16       90  

[3 rows x 286 columns]


This is a very large file. I will create a new DataFrame that only contains the columns I want to examine. 